In [4]:
import torch

In [5]:
torch.cuda.is_available()

True

## Datasets

In [6]:
class one_task():
    def __init__(self, path, data_name, batch_size, code, data_type='TPX'):
        '''
        data_name: 'five_minute' or 'hourly'
        data_type: 'TPX' (default) or 'MOEX'
        '''
        self.batch_size = batch_size
        self.data_name = data_name
        self.path = path
        self.code = code
        self.data_type = data_type
        if data_name != 'five_minute' and data_name != 'ten_minute':
            raise NotImplementedError

        train_dataset, dev_dataset, test_dataset = self.read_csv()
        if self.data_name == 'five_minute':
            self.trainloader = self.get_loader_five_minute(raw_data=train_dataset,
                                                           batch_size=self.batch_size, mode='train')
            self.devloader = self.get_loader_five_minute(raw_data=dev_dataset,
                                                         batch_size=self.batch_size, mode='dev')
            self.testloader = self.get_loader_five_minute(raw_data=test_dataset,
                                                          batch_size=self.batch_size, mode='test')
        elif self.data_name == 'ten_minute':
            self.trainloader = self.get_loader_ten_minute(raw_data=train_dataset,
                                                          batch_size=self.batch_size, mode='train')
            self.devloader = self.get_loader_ten_minute(raw_data=dev_dataset,
                                                        batch_size=self.batch_size, mode='dev')
            self.testloader = self.get_loader_ten_minute(raw_data=test_dataset,
                                                         batch_size=self.batch_size, mode='test')

    def read_csv(self):
        train_data, train_history, test_data, test_history = {}, set(), {}, set()
        finetune_data, finetune_history, dev_data, dev_history = {}, set(), {}, set()

        with open(self.path) as f:
            reader = csv.reader(f)
            if self.data_type == 'MOEX':
                for row in reader:
                    
                    if row[0] == '':
                        continue
    
                    time_stamp = datetime.datetime.strptime(row[2] + ' ' + row[6], "%Y-%m-%d %H:%M")
                    hours = time_stamp.hour
                    minutes = time_stamp.minute                 
                    chlotv = [float(row[1]), float(row[3]), float(row[4]), float(row[5]), float(hours * 60 + minutes), float(row[7]), float(row[8])]
                    
                    if self.data_name == 'five_minute':
                        if time_stamp.year > 2022:
                            continue
                        if time_stamp.year < 2022:
                            continue
                        if time_stamp.month < 1:
                            continue
                        if time_stamp.month > 2:
                            continue
    
                        # 2022 Jan 3 - 21 train
                        # 2022 Jan 22 - 26 dev
                        # 2022 Jan 27 - Feb 11 test

                        if datetime.datetime.strptime('2022-01-27', '%Y-%m-%d') <= time_stamp <= datetime.datetime.strptime('2022-02-11', '%Y-%m-%d'):
                            test_data[time_stamp] = chlotv
                            test_history.add(datetime.datetime.strptime(row[2], "%Y-%m-%d"))
                        else:
                            dev_data[time_stamp] = chlotv
                            dev_history.add(datetime.datetime.strptime(row[2], "%Y-%m-%d"))
    
                            train_data[time_stamp] = chlotv
                            train_history.add(datetime.datetime.strptime(row[2], "%Y-%m-%d"))
            
                        
            elif self.data_type == 'TPX':             
                for row in reader:
                    if row[0] == '':
                        continue
                    time_stamp = datetime.datetime.strptime(row[2] + ' ' + row[6], "%Y-%m-%d %H:%M")
    
                    hours = time_stamp.hour
                    minutes = time_stamp.minute
                    
                    chlotv = [float(row[1]), float(row[3]), float(row[4]), float(row[5]), float(hours * 60 + minutes), float(row[7])]
    
                    if self.data_name == 'five_minute':
                        if time_stamp.year > 2018:
                            continue
                        if time_stamp.year < 2017:
                            continue
                        if time_stamp.year == 2017 and time_stamp.month < 11:
                            continue
                        if time_stamp.year == 2018 and time_stamp.month > 2:
                            continue
    
                        # 2017 Dec 15 18 19 20 12 22 25 26
                        # dev 27 28 29
                        # 2018 Feb 1 2 5 test
    
                        if time_stamp.year == 2018:
                            test_data[time_stamp] = chlotv
                            test_history.add(datetime.datetime.strptime(row[2], "%Y-%m-%d"))
                        else:
                            dev_data[time_stamp] = chlotv
                            dev_history.add(datetime.datetime.strptime(row[2], "%Y-%m-%d"))
    
                            train_data[time_stamp] = chlotv
                            train_history.add(datetime.datetime.strptime(row[2], "%Y-%m-%d"))
    
                    elif self.data_name == 'ten_minute':
    
                        if time_stamp.year > 2018:
                            continue
                        if time_stamp.year < 2017:
                            continue
                        if time_stamp.year == 2017 and time_stamp.month < 6:
                            continue
                        if time_stamp.year == 2018 and time_stamp.month > 2:
                            continue
    
                        # 2017 8 9 10 11train
                        # 2017 12 dev
                        # 2018 2 test
    
                        if time_stamp.year == 2018:
                            test_data[time_stamp] = chlotv
                            test_history.add(datetime.datetime.strptime(row[2], "%Y-%m-%d"))
                        else:
                            if time_stamp.month > 9:
                                dev_data[time_stamp] = chlotv
                                dev_history.add(datetime.datetime.strptime(row[2], "%Y-%m-%d"))
    
                            if time_stamp.month < 12:
                                train_data[time_stamp] = chlotv
                                train_history.add(datetime.datetime.strptime(row[2], "%Y-%m-%d"))
                                
        return (train_data, train_history), (dev_data, dev_history), (test_data, test_history)

    def get_loader_five_minute(self, raw_data, batch_size, mode):

        d, history_set = raw_data
        dataloader = []
        if self.data_type == 'TPX': 
            num_features = 6
        else:
            num_features = 7
        
        for x in d:
            if mode == 'train':
                if self.data_type == 'TPX':                    
                    if (x.year < 2017):
                        continue
                    if (x.year == 2017) and (x.month != 12):
                        continue
                    if (x.year == 2017) and (x.month == 12) and (x.day < 15):
                        continue
                    if (x.year == 2017) and (x.month == 12) and (x.day > 26):
                        continue
                    train_mode = True
                
                elif self.data_type == 'MOEX':
                    if datetime.datetime.strptime('2022-01-03', '%Y-%m-%d') <= x <= datetime.datetime.strptime('2022-01-24', '%Y-%m-%d'):
                        train_mode = True
                    else:
                        continue
    
            elif mode == 'dev':
                if self.data_type == 'TPX':
                    
                    if (x.year < 2017):
                        continue
                    if (x.year == 2017) and (x.month != 12):
                        continue
                    if (x.year == 2017) and (x.month == 12) and (x.day < 27):
                        continue
                    if (x.year == 2017) and (x.month == 12) and (x.day > 29):
                        continue
                    train_mode = False
        
                elif self.data_type == 'MOEX':
                    if datetime.datetime.strptime('2022-01-25', '%Y-%m-%d') <= x <= datetime.datetime.strptime('2022-01-26', '%Y-%m-%d'):
                        train_mode = False
                    else:
                        continue
    
            elif mode == 'test':
                if self.data_type == 'TPX':
                    
                    if (x.year == 2018) and (x.month != 2):
                        continue
                    if (x.year == 2018) and (x.month == 2) and (x.day > 5):  
                        continue
                    train_mode = False
        
                elif self.data_type == 'MOEX':
                    if datetime.datetime.strptime('2022-01-27', '%Y-%m-%d') <= x <= datetime.datetime.strptime('2022-02-11', '%Y-%m-%d'):
                        train_mode = False
                    else:
                        continue
    
            else:
                raise NotImplementedError
                
            # TPX
            # 2017 Dec 15 18 19 20 21 22 25 26 train
            # 2017 Dec 27 28 29 dev
            # 2018 Feb 1 2 5 test
            
            tensor = torch.FloatTensor(1, 12, num_features)
            tensor[:, :, :] = -1
            
            test_time = x + datetime.timedelta(minutes=-5 * 12)
            if test_time not in d:
                continue

            con_flag = False
            for i in range(1, 13):
                y = x + datetime.timedelta(minutes=-5 * i)
                if y in d:
                    for j in range(num_features):
                        tensor[0, 12 - i, j] = d[y][j]
                else:
                    con_flag = True
                    break
            if con_flag:
                continue

            history = torch.FloatTensor(1, 10, num_features)
            history[:, :, :] = -1

            cnt = 0
            y = x
            while (cnt < 10):
                y = y + datetime.timedelta(days=-1)
                # print(datetime.datetime.strptime(str(y).split(' ')[0], "%Y-%m-%d") in history_set)
                if (datetime.datetime.strptime(str(y).split(' ')[0], "%Y-%m-%d") in history_set) \
                        or (y.year < x.year):
                    cnt += 1
                    if y in d:
                        for j in range(num_features):
                            history[0, 10 - cnt, j] = d[y][j]
                    else:
                        con_flag = True
                        break
            if con_flag:
                continue
            
            history.min().item()
            tensor.min().item()
            
            if (history.min().item() > -0.5) and (tensor.min().item() > -0.5):
                dataloader.append((tensor, history, torch.FloatTensor([[d[x][-1]]])))
        random.shuffle(dataloader)
        l = len(dataloader)
        print(l)
        testloader = []

        if l == 0:
            return testloader

        if train_mode == True:
            tensor = torch.cat([dataloader[j][0] for j in range(l)], dim=0)
            tensor_v20 = torch.cat([dataloader[j][1] for j in range(l)], dim=0)
            tensor_v = torch.cat([dataloader[j][2] for j in range(l)], dim=0)
            input_data = torch.cat([tensor.reshape(l, -1), tensor_v20.reshape(l, -1)], dim=1)
            input_data = torch.log(1 + input_data)
            tensor_v = torch.log(1 + tensor_v)
            testloader = (input_data, tensor_v)
        else:
            for i in range(l // batch_size):
                tensor = torch.cat([dataloader[j][0] for j in range(i * batch_size, (i + 1) * batch_size)], 0)
                tensor_v20 = torch.cat([dataloader[j][1] for j in range(i * batch_size, (i + 1) * batch_size)], 0)
                tensor_v = torch.cat([dataloader[j][2] for j in range(i * batch_size, (i + 1) * batch_size)], 0)

                input_data = torch.cat([tensor.reshape(batch_size, -1), tensor_v20.reshape(batch_size, -1)], dim=1)
                input_data = torch.log(1 + input_data)
                tensor_v = torch.log(1 + tensor_v)
                testloader.append((input_data, tensor_v))
            if l % batch_size != 0:
                tensor = torch.cat([dataloader[j][0] for j in range(-(l % batch_size), 0)], 0)
                tensor_v20 = torch.cat([dataloader[j][1] for j in range(-(l % batch_size), 0)], 0)
                tensor_v = torch.cat([dataloader[j][2] for j in range(-(l % batch_size), 0)], 0)

                input_data = torch.cat([tensor.reshape(l % batch_size, -1), tensor_v20.reshape(l % batch_size, -1)],
                                       dim=1)
                input_data = torch.log(1 + input_data)
                tensor_v = torch.log(1 + tensor_v)
                testloader.append((input_data, tensor_v))

        return testloader



    
    def get_loader_ten_minute(self, raw_data, batch_size, mode):

        d, history_set = raw_data
        dataloader = []
        for x in d:

            if mode == 'train':
                if (x.year < 2017):
                    continue
                if (x.year == 2017) and (x.month == 12):
                    continue
                if (x.year == 2017) and (x.month < 8):
                    continue
                train_mode = True

            elif mode == 'dev':
                if (x.year < 2017):
                    continue
                if (x.year == 2017) and (x.month != 12):
                    continue
                train_mode = False

            elif mode == 'test':
                if (x.year != 2018):
                    continue
                if (x.year == 2018) and (x.month != 2):
                    continue
                train_mode = False

            else:
                raise NotImplementedError

            test_time = x + datetime.timedelta(minutes=-5 * 25)
            if test_time not in d:
                continue

            if x.hour > 11:
                if x.hour < 14:
                    continue
                if x.hour == 14:
                    if x.minute < 35:
                        continue

            tensor = torch.FloatTensor(1, 25, 5)
            tensor[:, :, :] = -1

            con_flag = False
            for i in range(1, 26):
                y = x + datetime.timedelta(minutes=-5 * i)
                if y in d:
                    for j in range(5):
                        tensor[0, 25 - i, j] = d[y][j]
                else:
                    con_flag = True
                    break

            if con_flag:
                continue

            history1 = torch.FloatTensor(1, 10, 5)
            history1[:, :, :] = -1
            cnt = 0
            y = x
            while (cnt < 10):
                y = y + datetime.timedelta(days=-1)
                if (datetime.datetime.strptime(str(y).split(' ')[0], "%Y-%m-%d") in history_set) \
                        or (y.year < x.year):
                    cnt += 1
                    if y in d:
                        for j in range(5):
                            history1[0, 10 - cnt, j] = d[y][j]
                    else:
                        con_flag = True
                        break
            if con_flag:
                continue

            history2 = torch.FloatTensor(1, 10, 5)
            history2[:, :, :] = -1
            cnt = 0
            y = x + datetime.timedelta(minutes=-5)
            while (cnt < 10):
                y = y + datetime.timedelta(days=-1)
                if (datetime.datetime.strptime(str(y).split(' ')[0], "%Y-%m-%d") in history_set) or (y.year < x.year):
                    cnt += 1
                    if y in d:
                        for j in range(5):
                            history2[0, 10 - cnt, j] = d[y][j]
                    else:
                        con_flag = True
                        break
            if con_flag:
                continue

            if (history1.min().item() > -0.5) and (tensor.min().item() > -0.5) and (history2.min().item() > -0.5):
                dataloader.append(
                    self.process_ten_minute_data(history1, history2, tensor, torch.FloatTensor([[d[x][-1]]]))
                )

        random.shuffle(dataloader)
        l = len(dataloader)
        testloader = []

        if l == 0:
            return testloader

        if train_mode == True:
            tensor = torch.cat([dataloader[j][0] for j in range(l)], dim=0)
            tensor_v20 = torch.cat([dataloader[j][1] for j in range(l)], dim=0)
            tensor_v = torch.cat([dataloader[j][2] for j in range(l)], dim=0)
            input_data = torch.cat([tensor.reshape(l, -1), tensor_v20.reshape(l, -1)], dim=1)
            input_data = torch.log(1 + input_data)
            tensor_v = torch.log(1 + tensor_v)
            testloader = (input_data, tensor_v)

        else:
            for i in range(l // batch_size):
                tensor = torch.cat([dataloader[j][0] for j in range(i * batch_size, (i + 1) * batch_size)], 0)
                tensor_v20 = torch.cat([dataloader[j][1] for j in range(i * batch_size, (i + 1) * batch_size)], 0)
                tensor_v = torch.cat([dataloader[j][2] for j in range(i * batch_size, (i + 1) * batch_size)], 0)

                input_data = torch.cat([tensor.reshape(batch_size, -1), tensor_v20.reshape(batch_size, -1)], dim=1)
                input_data = torch.log(1 + input_data)
                tensor_v = torch.log(1 + tensor_v)
                testloader.append((input_data, tensor_v))
            if l % batch_size != 0:
                tensor = torch.cat([dataloader[j][0] for j in range(-(l % batch_size), 0)], 0)
                tensor_v20 = torch.cat([dataloader[j][1] for j in range(-(l % batch_size), 0)], 0)
                tensor_v = torch.cat([dataloader[j][2] for j in range(-(l % batch_size), 0)], 0)

                input_data = torch.cat([tensor.reshape(l % batch_size, -1), tensor_v20.reshape(l % batch_size, -1)],
                                       dim=1)
                input_data = torch.log(1 + input_data)
                tensor_v = torch.log(1 + tensor_v)
                testloader.append((input_data, tensor_v))

        return testloader

    def process_ten_minute_data(self, history1, history2, tensor, v):

        res_history = torch.FloatTensor(1, 10, 5)
        res_history[:, :, 0] = history1[:, :, 0]  # close
        res_history[:, :, 1] = torch.max(history1[:, :, 1], history2[:, :, 1])  # high
        res_history[:, :, 2] = torch.min(history1[:, :, 2], history2[:, :, 2])  # low
        res_history[:, :, 3] = history2[:, :, 3]  # open
        res_history[:, :, 4] = history1[:, :, 4] + history2[:, :, 4]  # volume

        res_v = v + tensor[0, -1, -1]

        res_tensor = torch.FloatTensor(1, 12, 5)
        for i in range(12):
            res_tensor[:, i, 0] = tensor[:, 2 * i + 1, 0]  # close
            res_tensor[:, i, 1] = torch.max(tensor[:, i * 2, 1], tensor[:, i * 2 + 1, 1])  # high
            res_tensor[:, i, 2] = torch.min(tensor[:, i * 2, 2], tensor[:, i * 2 + 1, 2])  # low
            res_tensor[:, i, 3] = tensor[:, 2 * i, 3]  # open
            res_tensor[:, i, 4] = tensor[:, 2 * i, 4] + tensor[:, 2 * i + 1, 4]  # volume

        return res_tensor, res_history, res_v


class Tasks():
    def __init__(self, data_name, filelist, dir_name='./raw_data', cache_dir='./smaller_cache', batch_size=32, data_type='TPX'):
        self.data_name = data_name
        self.filelist = filelist
        self.cache_dir = cache_dir
        self.dir_name = dir_name
        self.tasks = {}
        self.task_names = set()
        self.batch_size = batch_size
        self.data_type = data_type 

        self.get_all_tasks()

    def get_all_tasks(self):
        print("getting all tasks...")
        show_step = 10
        for idx, task_name in enumerate(self.filelist):
            path = task_name + '.csv'
            self.task_names.add(task_name)
            cache_name = os.path.join(self.cache_dir, task_name + '_' + self.data_name + '.pt')
            if os.path.exists(cache_name):
                self.tasks[task_name] = torch.load(cache_name)
            else:
                self.tasks[task_name] = one_task(path=os.path.join(self.dir_name, path), data_name=self.data_name, batch_size=self.batch_size, code=task_name, data_type=self.data_type)
                torch.save(self.tasks[task_name], cache_name)
                if (idx + 1) % show_step == 0:
                    print(f'step {idx + 1} finished!')
        print('finish getting tasks!')

    def checkdata(self):
        file_cnt = len(self.filelist)
        train_cnt, dev_cnt, test_cnt = 0, 0, 0
        train_zero, dev_zero, test_zero = 0, 0, 0
        train_total, dev_total, test_total = 0, 0, 0

        for i in self.filelist:
            if len(self.tasks[i].trainloader) > 0:
                train_len = self.tasks[i].trainloader[0].shape[0]
                train_total += train_len
            else:
                train_len = 0
            dev_len = 0
            test_len = 0

            if len(self.tasks[i].devloader) > 0:
                for x, y in self.tasks[i].devloader:
                    dev_len += x.shape[0]

            if len(self.tasks[i].testloader) > 0:
                for x, y in self.tasks[i].testloader:
                    test_len += x.shape[0]

            dev_total += dev_len
            test_total += test_len

            if train_len < 32:
                train_cnt += 1
                if train_len == 0:
                    train_zero += 1
            if test_len < 8:
                test_cnt += 1
                if len(self.tasks[i].testloader) == 0:
                    test_zero += 1

            if dev_len < 8:
                dev_cnt += 1
                if len(self.tasks[i].devloader) == 0:
                    dev_zero += 1
        print('total stock number:', file_cnt)
        print(train_total, train_cnt, train_zero)
        print(dev_total, dev_cnt, dev_zero)
        print(test_total, test_cnt, test_zero)

## Baseline models

In [7]:
import torch
import torch.nn as nn
import numpy as np
import random
import os
import pandas as pd
import copy
import torch.nn.functional as F
import math


class mlp(nn.Module):
    def __init__(self, emb_dim=132):
        super().__init__()
        self.fc = nn.Linear(in_features=emb_dim, out_features=1)
        pass

    def forward(self, x):
        '''
        x: [b,emb_dim]
        '''
        pred = self.fc(x)
        pred = pred.reshape((-1))
        return pred


class Attentive_Pooling(nn.Module):
    def __init__(self, hidden_size):
        super(Attentive_Pooling, self).__init__()
        self.w_1 = nn.Linear(hidden_size, hidden_size)
        self.u = nn.Linear(hidden_size, 1, bias=False)

    def forward(self, memory, mask=None):
        h = torch.tanh(self.w_1(memory))
        score = torch.squeeze(self.u(h), -1)  # node,
        if mask is not None:
            score = score.masked_fill(mask.eq(0), -1e9)
        alpha = F.softmax(score, -1)  # node,
        s = torch.sum(torch.unsqueeze(alpha, -1) * memory, -2)
        return s


class LSTM(nn.Module):
    def __init__(self, input_size, hidden_size, num_layer, feature_size, full_chlotv,
                 attn_pooling, num_features):
        super(LSTM, self).__init__()
        self.full_chlotv = full_chlotv
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.num_layer = num_layer
        self.feature_size = feature_size
        self.attn_pooling = attn_pooling
        self.num_features = num_features
        
        if self.full_chlotv:
            self.linear_chlotv = nn.Linear(num_features, input_size)
            self.linear_history = nn.Linear(num_features, input_size)
        else:
            self.linear_chlotv = nn.Linear(1, input_size)
            self.linear_history = nn.Linear(1, input_size)

        self.lstm_chlotv = nn.LSTM(input_size, hidden_size, num_layer, batch_first=True)
        self.lstm_history = nn.LSTM(input_size, hidden_size, num_layer, batch_first=True)

        if self.attn_pooling:
            self.attn_pooling_chlotv = Attentive_Pooling(hidden_size)
            self.attn_pooling_history = Attentive_Pooling(hidden_size)

        if feature_size != 0:
            self.linear_out1 = nn.Linear(2 * hidden_size, feature_size)
            self.linear_out2 = nn.Linear(feature_size, 1)
        else:
            self.linear_out = nn.Linear(2 * hidden_size, 1)

    def forward(self, x, early_exit=False):
        chlotv = x[:, :self.num_features*12].reshape((-1, 12, self.num_features))
        history = x[:, self.num_features*12:].reshape((-1, 10, self.num_features))
        self.lstm_chlotv.flatten_parameters()
        self.lstm_history.flatten_parameters()
        if self.full_chlotv:
            chlotv = F.relu(self.linear_chlotv(chlotv))
            history = F.relu(self.linear_history(history))
        else:
            chlotv = F.relu(self.linear_chlotv(chlotv[:, :, -1:]))
            history = F.relu(self.linear_history(history[:, :, -1:]))

        B, L, H = chlotv.size()
        chlotv, _ = self.lstm_chlotv(chlotv)
        history, _ = self.lstm_history(history)

        if self.attn_pooling:
            chlotv = self.attn_pooling_chlotv(chlotv)
            history = self.attn_pooling_history(history)
        else:
            chlotv = chlotv[:, -1, :]
            history = history[:, -1, :]

        x = torch.cat((chlotv, history), dim=1)

        if self.feature_size != 0:
            x = F.relu(self.linear_out1(x))
            if early_exit:
                return x
            x = self.linear_out2(x)
        else:
            x = self.linear_out(x)

        x = x.reshape((-1))
        return x


def clones(module, N):
    return nn.ModuleList([copy.deepcopy(module) for _ in range(N)])


class Transformer(nn.Module):
    def __init__(self, input_size=200, hidden_size=200, num_layer=6, num_features=7):
        super(Transformer, self).__init__()
        self.num_features = num_features
        c = copy.deepcopy
        d_model = hidden_size
        head_num = 8
        dropout = 0.1
        self.dropout = nn.Dropout(dropout)
        attn = MultiHeadedAttention(head_num, d_model, dropout)
        ff = PositionwiseFeedForward(d_model, d_model, dropout)
        self.position_emb = nn.Embedding(10 + 12 + 1, input_size)
        self.word_lienar = nn.Linear(num_features, input_size)
        self.emb_proj = nn.Linear(input_size, d_model)
        self.encoder = trm_Encoder(EncoderLayer(d_model, c(attn), c(ff), dropout), num_layer)
        self.w_out = nn.Linear(d_model, 1)
        self.bos_emb = nn.Parameter(torch.zeros(1, 1, num_features))
        self.init()

    def init(self):
        for p in self.parameters():
            if p.dim() > 1:
                nn.init.xavier_uniform_(p)
                

    def forward(self, x, early_exit=False):
        chlotv = x[:, :self.num_features*12].reshape(-1, 12, self.num_features)
        history = x[:, self.num_features*12:].reshape(-1, 10, self.num_features)

        batch_size, seq_len = chlotv.size(0), 10 + 12 + 1
        device = chlotv.device

        input = torch.cat((self.bos_emb.repeat(batch_size, 1, 1), chlotv, history), dim=1)
        bos = torch.ones(input.size(0), 1, device=device).long().fill_(2)
        word_mask = torch.ones(batch_size, seq_len, device=device).long()
        pos_indices = torch.unsqueeze(torch.arange(seq_len), 0).repeat(batch_size, 1).to(device)
        word_emb = self.word_lienar(input)
        pos_emb = self.position_emb(pos_indices)
        emb = self.emb_proj(word_emb + pos_emb)

        hidden = self.encoder(emb, word_mask)

        if early_exit:
            return hidden[:, 0, :]

        result = self.w_out(self.dropout(hidden[:, 0, :])).reshape((-1))

        return result


class LayerNorm(nn.Module):

    def __init__(self, features, eps=1e-6):
        super(LayerNorm, self).__init__()
        self.a_2 = nn.Parameter(torch.ones(features))
        self.b_2 = nn.Parameter(torch.zeros(features))
        self.eps = eps

    def forward(self, x):
        mean = x.mean(-1, keepdim=True)
        std = x.std(-1, keepdim=True)
        return self.a_2 * (x - mean) / (std + self.eps) + self.b_2


class trm_Encoder(nn.Module):

    def __init__(self, layer, N):
        super(trm_Encoder, self).__init__()
        self.layers = clones(layer, N)
        self.norm = LayerNorm(layer.size)

    def forward(self, x, mask):
        if mask.dim() == 2:
            seq_len = mask.size(1)
            mask = mask.unsqueeze(1).expand(-1, seq_len, -1)
            assert mask.size() == (x.size(0), seq_len, seq_len)
        for layer in self.layers:
            x = layer(x, mask)
        return self.norm(x)


class SublayerConnection(nn.Module):
    def __init__(self, size, dropout):
        super(SublayerConnection, self).__init__()
        self.norm = LayerNorm(size)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x, sublayer):
        return x + self.dropout(sublayer(self.norm(x)))


class EncoderLayer(nn.Module):

    def __init__(self, size, self_attn, feed_forward, dropout):
        super(EncoderLayer, self).__init__()
        self.self_attn = self_attn
        self.feed_forward = feed_forward
        self.sublayer = clones(SublayerConnection(size, dropout), 2)
        self.size = size

    def forward(self, x, mask):
        x = self.sublayer[0](x, lambda x: self.self_attn(x, x, x, mask))
        return self.sublayer[1](x, self.feed_forward)


def attention(query, key, value, mask=None, dropout=0.0):
    d_k = query.size(-1)
    scores = torch.matmul(query, key.transpose(-2, -1)) / math.sqrt(d_k)
    if mask is not None:
        scores = scores.masked_fill(mask.eq(0), -1e9)
    p_attn = F.softmax(scores, dim=-1)
    # (Dropout described below)
    p_attn = F.dropout(p_attn, p=dropout)
    return torch.matmul(p_attn, value), p_attn


class MultiHeadedAttention(nn.Module):
    def __init__(self, h, d_model, dropout=0.1):
        super(MultiHeadedAttention, self).__init__()
        assert d_model % h == 0
        # We assume d_v always equals d_k
        self.d_k = d_model // h
        self.h = h
        self.p = dropout
        self.linears = clones(nn.Linear(d_model, d_model), 4)  # clone linear for 4 times, query, key, value, output
        self.attn = None

    def forward(self, query, key, value, mask=None):
        if mask is not None:
            # Same mask applied to all h heads.
            mask = mask.unsqueeze(1)
            assert mask.dim() == 4  # batch, head, seq_len, seq_len
        nbatches = query.size(0)

        # 1) Do all the linear projections in batch from d_model => head * d_k
        query, key, value = [l(x).view(nbatches, -1, self.h, self.d_k).transpose(1, 2)
                             for l, x in zip(self.linears, (query, key, value))]

        # 2) Apply attention on all the projected vectors in batch.
        x, self.attn = attention(query, key, value, mask=mask, dropout=self.p)

        # 3) "Concat" using a view and apply a final linear.
        x = x.transpose(1, 2).contiguous().view(nbatches, -1, self.h * self.d_k)
        return self.linears[-1](x)


class PositionwiseFeedForward(nn.Module):

    def __init__(self, d_model, d_ff, dropout=0.1):
        super(PositionwiseFeedForward, self).__init__()
        # Torch linears have a `b` by default.
        self.w_1 = nn.Linear(d_model, d_ff)
        self.w_2 = nn.Linear(d_ff, d_model)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        return self.w_2(self.dropout(F.relu(self.w_1(x))))


class Embeddings(nn.Module):
    def __init__(self, emb_size, vocab, emb=None):
        super(Embeddings, self).__init__()
        if emb is not None:
            self.lut = emb
        else:
            self.lut = nn.Embedding(vocab, emb_size)
        self.emb_size = emb_size
        self.criterion = nn.CosineEmbeddingLoss(margin=0.5)

    def forward(self, x):
        return self.lut(x) * math.sqrt(self.emb_size)


class PositionalEncoding(nn.Module):

    def __init__(self, d_model, dropout, max_len=5000):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=dropout)

        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2) *
                             -(math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0)
        self.register_buffer('pe', pe)

    def forward(self, x):
        x = x + self.pe[:, :x.size(1)]
        return self.dropout(x)


## CNN LSTM

In [8]:
class CNN_LSTM(nn.Module):
    def __init__(self, input_size=200, hidden_size=200, num_layer=1, feature_size=30, 
                 full_chlotv=True, attn_pooling=True, kernel_size = 7, out_channels = 20, dropout_rate = 0.3, num_features=7):
        super(CNN_LSTM, self).__init__()
        self.full_chlotv = full_chlotv
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.num_layer = num_layer
        self.feature_size = feature_size
        self.attn_pooling = attn_pooling
        self.dropout_rate = dropout_rate
        self.num_features = num_features

        self.conv1_chlotv = nn.Sequential (nn.Conv1d(12, out_channels = out_channels, kernel_size=kernel_size, padding = 3),
            #nn.ReLU(),
            #nn.BatchNorm1d(norm_rate),
            #nn.Dropout(dropout_rate)
                                          ) 
        self.conv1_hist = nn.Sequential (nn.Conv1d(10, out_channels = out_channels, kernel_size=kernel_size,  padding = 3),
            #nn.ReLU(),
            #nn.BatchNorm1d(norm_rate),
            #nn.Dropout(dropout_rate)
                                        ) 

        if self.full_chlotv:
            self.linear_chlotv = nn.Linear(num_features, input_size)  
            self.linear_history = nn.Linear(num_features, input_size)
        else:
            self.linear_chlotv = nn.Linear(1, input_size)
            self.linear_history = nn.Linear(1, input_size)

        self.lstm_chlotv = nn.LSTM(input_size, hidden_size, num_layer, batch_first=True)
        self.lstm_history = nn.LSTM(input_size, hidden_size, num_layer, batch_first=True)

        if self.attn_pooling:
            self.attn_pooling_chlotv = Attentive_Pooling(hidden_size)
            self.attn_pooling_history = Attentive_Pooling(hidden_size)

        if feature_size != 0:
            self.linear_out1 = nn.Linear(2 * hidden_size, feature_size)
            self.linear_out2 = nn.Linear(feature_size, 1)
        else:
            self.linear_out = nn.Linear(2 * hidden_size, 1)

    def forward(self, x, early_exit=False):
        
        chlotv = x[:, :self.num_features*12].reshape((-1, 12, self.num_features))
        history = x[:, self.num_features*12:].reshape((-1, 10, self.num_features))

        if self.full_chlotv:
            chlotv = F.relu(self.linear_chlotv(chlotv))
            history = F.relu(self.linear_history(history))
        else:
            chlotv = F.relu(self.linear_chlotv(chlotv[:, :, -1:]))
            history = F.relu(self.linear_history(history[:, :, -1:]))

            
        chlotv = self.conv1_chlotv(chlotv)
        history = self.conv1_hist(history)

        self.lstm_chlotv.flatten_parameters()
        self.lstm_history.flatten_parameters()

        chlotv, _ = self.lstm_chlotv(chlotv)
        history, _ = self.lstm_history(history)

        if self.attn_pooling:
            chlotv = self.attn_pooling_chlotv(chlotv)
            history = self.attn_pooling_history(history)
        else:
            chlotv = chlotv[:, -1, :]
            history = history[:, -1, :]

        x = torch.cat((chlotv, history), dim=1)

        if self.feature_size != 0:
            x = F.relu(self.linear_out1(x))
            if early_exit:
                return x
            x = self.linear_out2(x)
        else:
            x = self.linear_out(x)

        x = x.reshape((-1))
        return x

## Utils

In [9]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from tqdm import tqdm
import numpy as np
import random
import argparse
import datetime
import csv
import math


def generate_train_data(train_tasks):
    print('Generating data...')
    input_x = []
    input_y = []

    for file_name in train_tasks.filelist:
        train_loader = train_tasks.tasks[file_name].trainloader
        x, y = train_loader

        input_x.append(x)
        input_y.append(y)

    input_x = torch.cat(input_x, dim=0)
    input_y = torch.cat(input_y, dim=0)
    print('Generating finished!')

    return input_x, input_y


def str2bool(v):
    if isinstance(v, bool):
        return v
    if v.lower() in ('yes', 'true', 't', 'y', '1'):
        return True
    elif v.lower() in ('no', 'false', 'f', 'n', '0'):
        return False
    else:
        raise argparse.ArgumentTypeError('Boolean value expected.')


def set_seed(seed):
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    np.random.seed(seed)
    random.seed(seed)


def print_running_info(args):
    print(f'current config: dataset:{args.dataset}, data_type:{args.data_type}, model type:{args.model_type}, '
          f'is baseline:{args.is_baseline}')

def read_filelist(filelist_name):
    train_list = []
    with open(filelist_name, 'r') as f:
        lines = f.readlines()
        for line in lines:
            train_list.append(line.strip())
    return train_list


def write_filelist(filelist_name, filelist):
    content = []
    for file in filelist:
        content.append(file + '\n')
    with open(filelist_name, 'w') as f:
        f.writelines(content)
    print(f"{len(content)} lines written.")


## Arguments

In [12]:

class Args:
  pass

args = Args()

# basics
args.log = '' # str
args.seed = 1 # int
args.gpu_id = '1' # str


# args.batch_size = 8 # int
args.max_epoch = 50 # int


# model
args.dataset = 'five_minute' # str
args.data_type = 'MOEX'
args.model_type = 'CNN_LSTM' # str
args.input_size = 300 # int
args.hidden_size = 128 # int
args.num_layer = 2 # int
args.feature_size = 30 # int
args.is_baseline = True # bool


## Main

In [14]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from tqdm import tqdm
import numpy as np
import random
import argparse
import datetime
import csv
import math



def train(model, epoch, batch_size, input_x, input_y, optimizer, is_linear=False):
    print('training...')
    model.train()
    train_loss = 0
    cnt = 0
    l = input_y.shape[0]
    order = list(range(l))
    random.shuffle(order)

    input_batch = []
    for i in range(l // batch_size):
        input_batch.append(
            (input_x[order[i * batch_size:(i + 1) * batch_size]], input_y[order[i * batch_size:(i + 1) * batch_size]])
        )

    for x1, y1 in input_batch:
        x1 = x1.cuda()
        y1 = y1.cuda()
        cnt += 1
        pred = model(x1)
        y1 = y1.reshape((-1))

        loss = torch.mean((pred - y1) ** 2) 
        train_loss += loss
        model.zero_grad()
        loss.backward()
        optimizer.step()

    train_loss /= cnt
    print('Train Epoch: {} \tMSE: {:.6f}'.format(epoch, train_loss), file=open(args.log, 'a'), flush=True)
    print('Train Epoch: {} \tMSE: {:.6f}'.format(epoch, train_loss))
    return train_loss

def test(model, device, test_tasks, test_list, is_linear=False):
    print('start testing')
    global args
    MSE, MAE, correct, cnt = 0, 0, 0, 0
    for file_name in test_list:
        model.eval()
        test_loader = test_tasks.tasks[file_name].testloader
        random.shuffle(test_loader)
        for x, y in test_loader:
            x = x.to(device)
            y = y.to(device)
            output = model(x)
            
            y = y.reshape((-1))
            b_size = x.shape[0]
            MSE += torch.sum(((output - y) ** 2), dim=0).item()
            MAE += torch.sum(((output - y).abs()), dim=0).item()
            correct += ((output - x[:, 59]) * (y - x[:, 59])).ge(0).float().sum().item()
            cnt += b_size
    MSE /= cnt
    MAE /= cnt
    correct /= cnt
    RMSE = math.sqrt(MSE)
    print('Test set: MSE: {:.6f}, RMSE: {:.6f}, MAE: {:.6f}, ACC: {:.6f} '.format(MSE, RMSE, MAE, correct),
          file=open(args.log, 'a'), flush=True)
    print('Test set: MSE: {:.6f}, RMSE: {:.6f}, MAE: {:.6f}, ACC: {:.6f} '.format(MSE, RMSE, MAE, correct), flush=True)
    return MSE, RMSE, MAE, correct


def valid(model, device, test_tasks, test_list, is_linear=False):
    print('start validation')
    global args
    MSE, cnt = 0, 0
    for file_name in test_list:
        model.eval()
        test_loader = test_tasks.tasks[file_name].devloader
        random.shuffle(test_loader)
        for x, y in test_loader:
            x = x.to(device)
            y = y.to(device)
            output = model(x)

            b_size = x.shape[0]
            y = y.reshape((-1))
            MSE += torch.sum(((output - y) ** 2), dim=0).item()
            cnt += b_size

    MSE /= cnt
    print('Valid set: MSE: {:.6f}'.format(MSE), file=open(args.log, 'a'), flush=True)
    print('Valid set: MSE: {:.6f}'.format(MSE), flush=True)
    return MSE



def main():

    os.environ['CUDA_VISIBLE_DEVICES'] = args.gpu_id
    set_seed(args.seed)

    print_running_info(args)

    train_list_dic = {'TPX': './filelist/TPX_stock_list.txt',
                      'MOEX': './filelist/MOEX_stock_list.txt'}
    cache_dir_dic = {'five_minute': './data/five_minute_cache'}

    res_save_path = './results/' + args.dataset + '_' + args.data_type + '_' + args.model_type + '_meta_result.csv'

    if args.log == '':
        args.log = datetime.datetime.now().strftime("log/%Y-%m-%d-%H%M%S.txt")
    else:
        print(args, file=open(args.log, 'a'), flush=True)


    train_list_filename = train_list_dic[args.data_type]
    train_list = read_filelist(train_list_filename)

    train_tasks = Tasks(data_name=args.dataset, filelist=train_list,
                        cache_dir=cache_dir_dic[args.dataset], batch_size=32, data_type = args.data_type)
    train_tasks.checkdata()

    test_list = train_tasks.filelist
    device = torch.device("cuda")

    if args.data_type == 'TPX':
        num_features = 6
    else:
        num_features = 7

    if args.model_type == 'linear':
        model = mlp()
    elif args.model_type == 'lstm':
        
        model = LSTM(input_size = 500,
    hidden_size = 200,
    num_layer = 3,
    feature_size = 30,
    full_chlotv = True,
    attn_pooling = True,
    num_features = num_features)
        
    elif args.model_type == 'transformer':
        model = Transformer(num_features = num_features)


    
    elif args.model_type == 'CNN_LSTM':
        model =  CNN_LSTM(input_size = 500,
                    hidden_size = 200,
                    num_layer = 3,
                    feature_size = 30,
                    full_chlotv = True,
                    attn_pooling = True,
                    kernel_size = 7,
                    out_channels = 80,
                    num_features = num_features)

        '''input_size = 500,
        hidden_size = 200,
        num_layer = 2,
        feature_size = 30,
        full_chlotv = True,
        attn_pooling = True,
        out_channels = 30,
        num_features = num_features setup for MOEX '''

    else:
        raise Exception('Invalid model type!')
    model = model.to(device)

    # training
    if args.model_type != 'linear' or args.is_baseline:
        if args.model_type == 'linear':
            is_linear = True
            optimizer = torch.optim.Adam(params=model.parameters(), lr=1e-4)
        else:
            is_linear = False
            optimizer = torch.optim.Adam(params=model.parameters(), lr=1e-4)
        baseline_res_save_path = './results/' + args.dataset + '_' + args.data_type + '_' + args.model_type+'_baseline_result.csv'

        max_valid_MSE, max_MSE, max_RMSE, max_MAE, max_ACC = 1e10, 0, 0, 0, 0

        input_x, input_y = generate_train_data(train_tasks)
        
        for epoch in range(1, args.max_epoch + 1):
            print(f'running epoch {epoch}')
            train(model, epoch=epoch, batch_size=32, input_x=input_x, input_y=input_y,
                      optimizer=optimizer, is_linear=is_linear)
 
            valid_MSE = valid(model, device, test_tasks=train_tasks, test_list=test_list, is_linear=is_linear)
            MSE, RMSE, MAE, ACC = test(model, device, test_tasks=train_tasks,
                                           test_list=test_list, is_linear=is_linear)
            if valid_MSE < max_valid_MSE:
                max_valid_MSE, max_MSE, max_RMSE, max_MAE, max_ACC = valid_MSE, MSE, RMSE, MAE, ACC
                model.cpu()
                torch.save(model, args.log.replace('.txt', '.pt'))
                model.cuda()

        f = open(baseline_res_save_path, 'a', encoding='utf-8')
        csv_writer = csv.writer(f)
        csv_writer.writerow(
            [args.dataset, args.data_type, args.model_type, args.seed, max_MSE, max_RMSE, max_MAE, max_ACC, max_valid_MSE])
        f.close()

        if args.is_baseline:
            return

        model = torch.load(args.log.replace('.txt', '.pt'))
        model = model.to(device)

 
main()




current config: dataset:five_minute, data_type:MOEX, model type:CNN_LSTM, is baseline:True
getting all tasks...
384
128
128
384
128
128
384
128
128
384
128
128
384
128
128
384
128
128
384
128
128
384
128
126
384
128
128
384
128
128
step 10 finished!
finish getting tasks!
total stock number: 10
3840 0 0
1280 0 0
1278 0 0
Generating data...
Generating finished!
running epoch 1
training...
Train Epoch: 1 	MSE: 57.302036
start validation
Valid set: MSE: 5.985436
start testing
Test set: MSE: 6.022319, RMSE: 2.454041, MAE: 2.068240, ACC: 0.752739 
running epoch 2
training...
Train Epoch: 2 	MSE: 5.658383
start validation
Valid set: MSE: 5.846581
start testing
Test set: MSE: 6.578206, RMSE: 2.564801, MAE: 2.211430, ACC: 0.752739 
running epoch 3
training...
Train Epoch: 3 	MSE: 5.618091
start validation
Valid set: MSE: 5.839280
start testing
Test set: MSE: 6.629371, RMSE: 2.574757, MAE: 2.221983, ACC: 0.752739 
running epoch 4
training...
Train Epoch: 4 	MSE: 5.261939
start validation
Valid s

## Naive forecasting

In [7]:
import pandas as pd
import numpy as np
import os
from sklearn.metrics import mean_squared_error, mean_absolute_error

directory = 'TPX'

metric_values = {
    'mse_moving_10_day': [],
    'rmse_moving_10_day': [],
    'mae_moving_10_day': [],
    'accuracy_moving_10_day': [],
    'mse_moving_1_hour': [],
    'rmse_moving_1_hour': [],
    'mae_moving_1_hour': [],
    'accuracy_moving_1_hour': [],
    'mse_yesterday': [],
    'rmse_yesterday': [],
    'mae_yesterday': [],
    'accuracy_yesterday': [],
    'mse_slot': [],
    'rmse_slot': [],
    'mae_slot': [],
    'accuracy_slot': [],
}

def calculate_direction_accuracy(actual, predicted):
    actual_change = np.sign(actual.pct_change())
    predicted_change = np.sign(predicted.pct_change())
    correct_predictions = (actual_change == predicted_change).sum()
    total_predictions = len(actual_change)
    return correct_predictions / total_predictions

for filename in os.listdir(directory):
    if filename.endswith('.csv'):

        df = pd.read_csv(os.path.join(directory, filename))
        
        df = df.replace(0, np.nan)
        df = df.dropna()
        


        df = df.sort_values('date')
        df['log_v'] = np.log(df['v'])

        df['log_moving_avg_10_day'] = df['log_v'].rolling(window=1410).mean()
        df['log_moving_avg_1_hour'] = df['log_v'].rolling(window=12).mean()

        df['log_moving_10_day'] = df['log_moving_avg_10_day'].shift(1410)
        df['log_moving_1_hour'] = df['log_moving_avg_1_hour'].shift(12)
        
        df['log_yesterday'] = df['log_v'].shift(141)
        df['log_slot'] = df['log_v'].shift(1)
        
        if directory == 'TPX':
            df_filtered = df[df['date'].isin(['2018-02-01', '2018-02-02', '2018-02-05'])]
        else:
            df_filtered = df[df['date'].isin(['2022-02-10', '2022-02-11'])]
                    
        # TPX 2018 Feb 1 2 5 test

        for forecast in ['moving_10_day', 'moving_1_hour', 'yesterday', 'slot']:
            mse = mean_squared_error(df_filtered['log_v'].dropna(), df_filtered['log_'+forecast].dropna())
            rmse = np.sqrt(mse)
            mae = mean_absolute_error(df_filtered['log_v'].dropna(), df_filtered['log_'+forecast].dropna())
            accuracy = calculate_direction_accuracy(df_filtered['log_v'], df_filtered['log_' + forecast])

            metric_values['mse_'+forecast].append(mse)
            metric_values['rmse_'+forecast].append(rmse)
            metric_values['mae_'+forecast].append(mae)
            metric_values['accuracy_'+forecast].append(accuracy)

mean_metrics = {metric: np.mean(values) for metric, values in metric_values.items()}


for metric, value in mean_metrics.items():
    print(f'Mean {metric.upper()}: {value}')

Mean MSE_MOVING_10_DAY: 1.1916975721166891
Mean RMSE_MOVING_10_DAY: 1.074903234672542
Mean MAE_MOVING_10_DAY: 0.8385429555939933
Mean ACCURACY_MOVING_10_DAY: 0.477185849776422
Mean MSE_MOVING_1_HOUR: 1.209998044925309
Mean RMSE_MOVING_1_HOUR: 1.082065785230775
Mean MAE_MOVING_1_HOUR: 0.8294724113734986
Mean ACCURACY_MOVING_1_HOUR: 0.48442277450309623
Mean MSE_YESTERDAY: 2.2275335570448958
Mean RMSE_YESTERDAY: 1.4730917545267483
Mean MAE_YESTERDAY: 1.1414292058428512
Mean ACCURACY_YESTERDAY: 0.4723970770663083
Mean MSE_SLOT: 1.6298355433639569
Mean RMSE_SLOT: 1.2483828416008946
Mean MAE_SLOT: 0.9090012206544753
Mean ACCURACY_SLOT: 0.3279136353348857


## Optimize

In [71]:
    os.environ['CUDA_VISIBLE_DEVICES'] = args.gpu_id
    set_seed(args.seed)

    print_running_info(args)

    train_list_dic = {'TPX': './filelist/TPX_stock_list.txt',
                      'MOEX': './filelist/MOEX_stock_list.txt'}
    cache_dir_dic = {'five_minute': './data/five_minute_cache'}

    res_save_path = './results/' + args.dataset + '_' + args.data_type + '_' + args.model_type + '_meta_result.csv'

    if args.log == '':
        args.log = datetime.datetime.now().strftime("log/%Y-%m-%d-%H%M%S.txt")
    else:
        print(args, file=open(args.log, 'a'), flush=True)

    # read data
    train_list_filename = train_list_dic[args.data_type]
    train_list = read_filelist(train_list_filename)

    train_tasks = Tasks(data_name=args.dataset, filelist=train_list,
                        cache_dir=cache_dir_dic[args.dataset], batch_size=32, data_type = args.data_type)
    train_tasks.checkdata()

    test_list = train_tasks.filelist
    device = torch.device("cuda")

current config: dataset:five_minute, data_type:TPX, model type:CNN_LSTM, is baseline:True
getting all tasks...
finish getting tasks!
total stock number: 40
8738 5 0
2930 7 2
3462 2 0


In [84]:
import optuna


def objective(trial):

    input_size = trial.suggest_int('input_size', 200, 500)
    hidden_size = trial.suggest_int('hidden_size', 100, 300)
    feature_size = trial.suggest_int('feature_size', 10, 50)
    attn_pooling = trial.suggest_categorical('attn_pooling', [True, False])
    num_layer= trial.suggest_int('num_layer', 2, 5)
    out_channels = trial.suggest_int('out_channels', 1, 100)
    

    model = CNN_LSTM(input_size = 500,
    hidden_size = 200,
    num_layer = num_layer,
    feature_size = 30,
    full_chlotv = True,
    attn_pooling = attn_pooling,
    out_channels = out_channels,
    num_features = 6
    )

    model = model.to(device)
    
    optimizer = torch.optim.Adam(params=model.parameters(), lr=5e-5)

    max_valid_MSE, max_MSE, max_RMSE, max_MAE, max_ACC = 1e10, 0, 0, 0, 0
    input_x, input_y = generate_train_data(train_tasks)

    for epoch in range(1, args.max_epoch + 1):
        print(f'running epoch {epoch}')
        train(model, epoch=epoch, batch_size=32, input_x=input_x, input_y=input_y,
                      optimizer=optimizer, is_linear=False)
            # scheduler.step()
        valid_MSE = valid(model, device, test_tasks=train_tasks, test_list=test_list, is_linear=False)
        MSE, RMSE, MAE, ACC = test(model, device, test_tasks=train_tasks,
                                           test_list=test_list, is_linear=False)
        if valid_MSE < max_valid_MSE:
            max_valid_MSE, max_MSE, max_RMSE, max_MAE, max_ACC = valid_MSE, MSE, RMSE, MAE, ACC
    return MSE



In [85]:
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=10)

best_trial = study.best_trial

print(f"Значение: {best_trial.value}")
print("Параметры: ")
for key, value in best_trial.params.items():
    print(f"    {key}: {value}")

[I 2024-05-26 13:38:05,587] A new study created in memory with name: no-name-599c35de-3777-412a-95d9-749d1b80839a


Generating data...
Generating finished!
running epoch 1
training...
Train Epoch: 1 	MSE: 26.767506
start validation
Valid set: MSE: 2.970137
start testing
Test set: MSE: 3.212101, RMSE: 1.792234, MAE: 1.375221, ACC: 0.577123 
running epoch 2
training...
Train Epoch: 2 	MSE: 3.057301
start validation
Valid set: MSE: 2.993838
start testing
Test set: MSE: 3.120507, RMSE: 1.766496, MAE: 1.354458, ACC: 0.578856 
running epoch 3
training...
Train Epoch: 3 	MSE: 3.055405
start validation
Valid set: MSE: 2.998716
start testing
Test set: MSE: 3.101815, RMSE: 1.761197, MAE: 1.350321, ACC: 0.581167 
running epoch 4
training...
Train Epoch: 4 	MSE: 3.043016
start validation
Valid set: MSE: 2.959171
start testing
Test set: MSE: 3.061957, RMSE: 1.749845, MAE: 1.340179, ACC: 0.580300 
running epoch 5
training...
Train Epoch: 5 	MSE: 2.091350
start validation
Valid set: MSE: 1.217667
start testing
Test set: MSE: 1.200089, RMSE: 1.095486, MAE: 0.820340, ACC: 0.625361 
running epoch 6
training...
Train 

[I 2024-05-26 13:49:01,582] Trial 0 finished with value: 0.560597267851948 and parameters: {'input_size': 441, 'hidden_size': 195, 'feature_size': 37, 'attn_pooling': False, 'num_layer': 3, 'out_channels': 76}. Best is trial 0 with value: 0.560597267851948.


Generating data...
Generating finished!
running epoch 1
training...
Train Epoch: 1 	MSE: 33.296360
start validation
Valid set: MSE: 3.455986
start testing
Test set: MSE: 4.575281, RMSE: 2.138991, MAE: 1.697308, ACC: 0.544483 
running epoch 2
training...
Train Epoch: 2 	MSE: 3.164404
start validation
Valid set: MSE: 3.005587
start testing
Test set: MSE: 3.098718, RMSE: 1.760318, MAE: 1.349652, ACC: 0.581456 
running epoch 3
training...
Train Epoch: 3 	MSE: 3.058109
start validation
Valid set: MSE: 3.022455
start testing
Test set: MSE: 3.071625, RMSE: 1.752605, MAE: 1.343853, ACC: 0.581167 
running epoch 4
training...
Train Epoch: 4 	MSE: 3.058867
start validation
Valid set: MSE: 3.001864
start testing
Test set: MSE: 3.105767, RMSE: 1.762319, MAE: 1.351216, ACC: 0.581167 
running epoch 5
training...
Train Epoch: 5 	MSE: 3.057580
start validation
Valid set: MSE: 3.006722
start testing
Test set: MSE: 3.096403, RMSE: 1.759660, MAE: 1.349149, ACC: 0.581456 
running epoch 6
training...
Train 

[I 2024-05-26 13:58:30,666] Trial 1 finished with value: 0.6117516813906267 and parameters: {'input_size': 346, 'hidden_size': 196, 'feature_size': 11, 'attn_pooling': False, 'num_layer': 4, 'out_channels': 44}. Best is trial 0 with value: 0.560597267851948.


...

Generating data...
Generating finished!
running epoch 1
training...
Train Epoch: 1 	MSE: 29.245369
start validation
Valid set: MSE: 2.998370
start testing
Test set: MSE: 3.529658, RMSE: 1.878738, MAE: 1.451079, ACC: 0.567880 
running epoch 2
training...
Train Epoch: 2 	MSE: 3.066214
start validation
Valid set: MSE: 3.006507
start testing
Test set: MSE: 3.068554, RMSE: 1.751729, MAE: 1.343057, ACC: 0.581456 
running epoch 3
training...
Train Epoch: 3 	MSE: 2.943716
start validation
Valid set: MSE: 2.426283
start testing
Test set: MSE: 2.750786, RMSE: 1.658549, MAE: 1.260114, ACC: 0.584344 
running epoch 4
training...
Train Epoch: 4 	MSE: 1.528573
start validation
Valid set: MSE: 1.128949
start testing
Test set: MSE: 1.069898, RMSE: 1.034359, MAE: 0.767016, ACC: 0.660312 
running epoch 5
training...
Train Epoch: 5 	MSE: 0.978250
start validation
Valid set: MSE: 0.908754
start testing
Test set: MSE: 0.796719, RMSE: 0.892591, MAE: 0.660999, ACC: 0.713172 
running epoch 6
training...
Train 

[I 2024-05-26 15:08:26,898] Trial 8 finished with value: 0.5901794176030063 and parameters: {'input_size': 384, 'hidden_size': 283, 'feature_size': 17, 'attn_pooling': True, 'num_layer': 2, 'out_channels': 37}. Best is trial 0 with value: 0.560597267851948.


Generating data...
Generating finished!
running epoch 1
training...
Train Epoch: 1 	MSE: 32.388557
start validation
Valid set: MSE: 2.989577
start testing
Test set: MSE: 3.133191, RMSE: 1.770082, MAE: 1.357325, ACC: 0.579723 
running epoch 2
training...
Train Epoch: 2 	MSE: 3.056252
start validation
Valid set: MSE: 2.978722
start testing
Test set: MSE: 3.165674, RMSE: 1.779234, MAE: 1.364647, ACC: 0.577412 
running epoch 3
training...
Train Epoch: 3 	MSE: 3.057254
start validation
Valid set: MSE: 2.977908
start testing
Test set: MSE: 3.161912, RMSE: 1.778177, MAE: 1.363826, ACC: 0.577412 
running epoch 4
training...
Train Epoch: 4 	MSE: 3.053108
start validation
Valid set: MSE: 3.009817
start testing
Test set: MSE: 3.063227, RMSE: 1.750208, MAE: 1.341885, ACC: 0.581167 
running epoch 5
training...
Train Epoch: 5 	MSE: 2.616707
start validation
Valid set: MSE: 1.342089
start testing
Test set: MSE: 1.332635, RMSE: 1.154398, MAE: 0.859852, ACC: 0.659445 
running epoch 6
training...
Train 

[I 2024-05-26 15:14:19,533] Trial 9 finished with value: 0.5788255152435127 and parameters: {'input_size': 231, 'hidden_size': 127, 'feature_size': 13, 'attn_pooling': True, 'num_layer': 5, 'out_channels': 17}. Best is trial 0 with value: 0.560597267851948.


Значение: 0.560597267851948
Параметры: 
    input_size: 441
    hidden_size: 195
    feature_size: 37
    attn_pooling: False
    num_layer: 3
    out_channels: 76
